In [2]:
# copy from mnist of exmaple in chainer v4.0
from __future__ import print_function

import argparse

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions


In [25]:
import os

In [29]:
# Network definition
BATCH_SIZE = 100
class SepLinear(chainer.ChainList):
    def __init__(self, n_in, n_unit, n_separate, input_sep_flag=True):
        super(SepLinear, self).__init__()
        with self.init_scope():
            for i in range(n_separate):
                self.add_link(L.Linear(n_in, n_unit))
        self.input_sep_flag = input_sep_flag
        self.n_separate = n_separate
    
    def __call__(self, x):
        if self.input_sep_flag:
            act_var = []
            for i in range(self.n_separate):
                var = F.expand_dims(x.reshape(BATCH_SIZE, self.n_separate, -1)[:, i, :], 1)
                act_var.append(self[i](var))
        else:
            act_var = []
            for i in range(self.n_separate):
                act_var.append(self[i](x))
        res = F.concat(act_var, axis=1)
        return res
            

class SepMLP(chainer.Chain):

    def __init__(self, n_units, n_out, separate_layer=(1, 2)):
        super(SepMLP, self).__init__()
        n_sep = n_out
        with self.init_scope():
            if 0 in separate_layer:
                self.l1 = SepLinear(None, n_units//n_sep, n_sep, False)
            else:
                self.l1 = L.Linear(None, n_units)
            
            if 1 in separate_layer:
                if 0 in separate_layer:
                    flag = True
                else:
                    flag = False
                self.l2 = SepLinear(None, n_units//n_sep, n_sep, flag)
            else:
                self.l2 = L.Linear(None, n_units)

            if 2 in separate_layer:
                if 1 in separate_layer:
                    flag = True
                else:
                    flag = False
                self.l3 = SepLinear(None, n_out//n_sep, n_sep, flag)
            else:
                self.l3 = L.Linear(None, n_out)

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)


In [35]:
def main():
#     parser = argparse.ArgumentParser(description='Chainer example: MNIST')
#     parser.add_argument('--batchsize', '-b', type=int, default=100,
#                         help='Number of images in each mini-batch')
#     parser.add_argument('--epoch', '-e', type=int, default=20,
#                         help='Number of sweeps over the dataset to train')
#     parser.add_argument('--frequency', '-f', type=int, default=-1,
#                         help='Frequency of taking a snapshot')
#     parser.add_argument('--gpu', '-g', type=int, default=-1,
#                         help='GPU ID (negative value indicates CPU)')
#     parser.add_argument('--out', '-o', default='result',
#                         help='Directory to output the result')
#     parser.add_argument('--resume', '-r', default='',
#                         help='Resume the training from snapshot')
#     parser.add_argument('--unit', '-u', type=int, default=1000,
#                         help='Number of units')
#     args = parser.parse_args()
    args_batchsize = BATCH_SIZE
    args_epoch = 50
    args_frequency = -1
    args_gpu = 0
    args_out = 'result'
    args_unit = 500

    if not os.path.exists(args_out):
        os.makedirs(args_out)
    print('GPU: {}'.format(args_gpu))
    print('# unit: {}'.format(args_unit))
    print('# Minibatch-size: {}'.format(args_batchsize))
    print('# epoch: {}'.format(args_epoch))
    print('')

    # Set up a neural network to train
    # Classifier reports softmax cross entropy loss and accuracy at every
    # iteration, which will be used by the PrintReport extension below.
    model = L.Classifier(SepMLP(args_unit, 10, separate_layer=(None,)))
    if args_gpu >= 0:
        # Make a specified GPU current
        chainer.backends.cuda.get_device_from_id(args_gpu).use()
        model.to_gpu()  # Copy the model to the GPU

    # Setup an optimizer
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    # Load the MNIST dataset
    train, test = chainer.datasets.get_mnist()

    train_iter = chainer.iterators.SerialIterator(train, args_batchsize)
    test_iter = chainer.iterators.SerialIterator(test, args_batchsize,
                                                 repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.updaters.StandardUpdater(
        train_iter, optimizer, device=args_gpu)
    trainer = training.Trainer(updater, (args_epoch, 'epoch'), out=args_out)

    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=args_gpu))

    # Dump a computational graph from 'loss' variable at the first iteration
    # The "main" refers to the target link of the "main" optimizer.
    trainer.extend(extensions.dump_graph('main/loss'))

    # Take a snapshot for each specified epoch
    frequency = args_epoch if args_frequency == -1 else max(1, args_frequency)
    trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

    trainer.extend(extensions.LogReport())
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                              'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
            'epoch', file_name='accuracy.png'))

    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    trainer.extend(extensions.ProgressBar())


    # Run the training
    chainer.serializers.save_npz(args_out +'/vanilla-base.model', model)
    trainer.run()
    chainer.serializers.save_npz(args_out +'/vanilla.model', model)
    
    print('sep 2')
    args_out = 'result2'
    if not os.path.exists(args_out):
        os.makedirs(args_out)

    model = L.Classifier(SepMLP(args_unit, 10, separate_layer=(2,)))
    if args_gpu >= 0:
        # Make a specified GPU current
        chainer.backends.cuda.get_device_from_id(args_gpu).use()
        model.to_gpu()  # Copy the model to the GPU

    # Setup an optimizer
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    # Load the MNIST dataset
    train, test = chainer.datasets.get_mnist()

    train_iter = chainer.iterators.SerialIterator(train, args_batchsize)
    test_iter = chainer.iterators.SerialIterator(test, args_batchsize,
                                                 repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.updaters.StandardUpdater(
        train_iter, optimizer, device=args_gpu)
    trainer = training.Trainer(updater, (args_epoch, 'epoch'), out=args_out)

    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=args_gpu))

    # Dump a computational graph from 'loss' variable at the first iteration
    # The "main" refers to the target link of the "main" optimizer.
    trainer.extend(extensions.dump_graph('main/loss'))

    # Take a snapshot for each specified epoch
    frequency = args_epoch if args_frequency == -1 else max(1, args_frequency)
    trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

    trainer.extend(extensions.LogReport())
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                              'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
            'epoch', file_name='accuracy.png'))

    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    trainer.extend(extensions.ProgressBar())


    # Run the training
    chainer.serializers.save_npz(args_out +'/sep2-base.model', model)
    trainer.run()
    chainer.serializers.save_npz(args_out +'/sep2.model', model)
    
    print('sep 1, 2')
    args_out = 'result1_2'
    if not os.path.exists(args_out):
        os.makedirs(args_out)

    model = L.Classifier(SepMLP(args_unit, 10, separate_layer=(1,2)))
    if args_gpu >= 0:
        # Make a specified GPU current
        chainer.backends.cuda.get_device_from_id(args_gpu).use()
        model.to_gpu()  # Copy the model to the GPU

    # Setup an optimizer
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)

    # Load the MNIST dataset
    train, test = chainer.datasets.get_mnist()

    train_iter = chainer.iterators.SerialIterator(train, args_batchsize)
    test_iter = chainer.iterators.SerialIterator(test, args_batchsize,
                                                 repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.updaters.StandardUpdater(
        train_iter, optimizer, device=args_gpu)
    trainer = training.Trainer(updater, (args_epoch, 'epoch'), out=args_out)

    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=args_gpu))

    # Dump a computational graph from 'loss' variable at the first iteration
    # The "main" refers to the target link of the "main" optimizer.
    trainer.extend(extensions.dump_graph('main/loss'))

    # Take a snapshot for each specified epoch
    frequency = args_epoch if args_frequency == -1 else max(1, args_frequency)
    trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

    trainer.extend(extensions.LogReport())
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                              'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
            'epoch', file_name='accuracy.png'))

    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    trainer.extend(extensions.ProgressBar())


    # Run the training
    chainer.serializers.save_npz(args_out +'/sep1_2-base.model', model)
    trainer.run()
    chainer.serializers.save_npz(args_out +'/sep1_2.model', model)

In [36]:
main()

GPU: 0
# unit: 500
# Minibatch-size: 100
# epoch: 50

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [..................................................]  0.33%
this epoch [########..........................................] 16.67%
       100 iter, 0 epoch / 50 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 50 epochs
    282.49 iters/sec. Estimated time to finish: 0:01:45.489754.
     total [..................................................]  1.00%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 50 epochs
    281.86 iters/sec. Estimated time to finish: 0:01:45.372091.
     total [..................................................]  1.33%
this epoch [#################################........

     total [#####.............................................] 10.67%
this epoch [################..................................] 33.33%
      3200 iter, 5 epoch / 50 epochs
     239.4 iters/sec. Estimated time to finish: 0:01:51.947955.
     total [#####.............................................] 11.00%
this epoch [#########################.........................] 50.00%
      3300 iter, 5 epoch / 50 epochs
    240.52 iters/sec. Estimated time to finish: 0:01:51.007947.
     total [#####.............................................] 11.33%
this epoch [#################################.................] 66.67%
      3400 iter, 5 epoch / 50 epochs
    241.52 iters/sec. Estimated time to finish: 0:01:50.134450.
     total [#####.............................................] 11.67%
this epoch [#########################################.........] 83.33%
      3500 iter, 5 epoch / 50 epochs
    242.32 iters/sec. Estimated time to finish: 0:01:49.358809.
6           0.0235795   0.07

     total [##########........................................] 21.00%
this epoch [#########################.........................] 50.00%
      6300 iter, 10 epoch / 50 epochs
    239.22 iters/sec. Estimated time to finish: 0:01:39.071274.
     total [##########........................................] 21.33%
this epoch [#################################.................] 66.67%
      6400 iter, 10 epoch / 50 epochs
    239.74 iters/sec. Estimated time to finish: 0:01:38.438566.
     total [##########........................................] 21.67%
this epoch [#########################################.........] 83.33%
      6500 iter, 10 epoch / 50 epochs
    240.33 iters/sec. Estimated time to finish: 0:01:37.781820.
11          0.0119797   0.0775088             0.996115       0.9832                    27.4983       
     total [###########.......................................] 22.00%
this epoch [..................................................]  0.00%
      6600 iter, 11 epoc

     total [###############...................................] 31.33%
this epoch [#################################.................] 66.67%
      9400 iter, 15 epoch / 50 epochs
    240.07 iters/sec. Estimated time to finish: 0:01:25.806984.
     total [###############...................................] 31.67%
this epoch [#########################################.........] 83.33%
      9500 iter, 15 epoch / 50 epochs
    240.45 iters/sec. Estimated time to finish: 0:01:25.257619.
16          0.0100109   0.0886808             0.996633       0.9834                    39.9681       
     total [################..................................] 32.00%
this epoch [..................................................]  0.00%
      9600 iter, 16 epoch / 50 epochs
    238.64 iters/sec. Estimated time to finish: 0:01:25.484304.
     total [################..................................] 32.33%
this epoch [########..........................................] 16.67%
      9700 iter, 16 epoc

     total [####################..............................] 41.67%
this epoch [#########################################.........] 83.33%
     12500 iter, 20 epoch / 50 epochs
    240.21 iters/sec. Estimated time to finish: 0:01:12.852847.
21          0.00672094  0.102893              0.997582       0.9808                    52.5239       
     total [#####################.............................] 42.00%
this epoch [..................................................]  0.00%
     12600 iter, 21 epoch / 50 epochs
    238.15 iters/sec. Estimated time to finish: 0:01:13.063287.
     total [#####################.............................] 42.33%
this epoch [########..........................................] 16.67%
     12700 iter, 21 epoch / 50 epochs
    238.12 iters/sec. Estimated time to finish: 0:01:12.651526.
     total [#####################.............................] 42.67%
this epoch [################..................................] 33.33%
     12800 iter, 21 epoc

...

     total [#############################.....................] 58.67%
this epoch [################..................................] 33.33%
     17600 iter, 29 epoch / 50 epochs
    46.036 iters/sec. Estimated time to finish: 0:04:29.352610.
     total [#############################.....................] 59.00%
this epoch [#########################.........................] 50.00%
     17700 iter, 29 epoch / 50 epochs
    46.034 iters/sec. Estimated time to finish: 0:04:27.192353.
     total [#############################.....................] 59.33%
this epoch [#################################.................] 66.67%
     17800 iter, 29 epoch / 50 epochs
     46.19 iters/sec. Estimated time to finish: 0:04:24.123977.
     total [#############################.....................] 59.67%
this epoch [#########################################.........] 83.33%
     17900 iter, 29 epoch / 50 epochs
    46.193 iters/sec. Estimated time to finish: 0:04:21.944515.
30          0.00449852  

     total [##################################................] 69.00%
this epoch [#########################.........................] 50.00%
     20700 iter, 34 epoch / 50 epochs
    46.111 iters/sec. Estimated time to finish: 0:03:21.688523.
     total [##################################................] 69.33%
this epoch [#################################.................] 66.67%
     20800 iter, 34 epoch / 50 epochs
    46.262 iters/sec. Estimated time to finish: 0:03:18.867644.
     total [##################################................] 69.67%
this epoch [#########################################.........] 83.33%
     20900 iter, 34 epoch / 50 epochs
     46.26 iters/sec. Estimated time to finish: 0:03:16.715035.
35          0.00255568  0.111837              0.999083       0.982                     454.6         
     total [###################################...............] 70.00%
this epoch [..................................................]  0.00%
     21000 iter, 35 epoc

     total [#######################################...........] 79.33%
this epoch [#################################.................] 66.67%
     23800 iter, 39 epoch / 50 epochs
    46.244 iters/sec. Estimated time to finish: 0:02:14.070180.
     total [#######################################...........] 79.67%
this epoch [#########################################.........] 83.33%
     23900 iter, 39 epoch / 50 epochs
    46.245 iters/sec. Estimated time to finish: 0:02:11.904950.
40          0.00236188  0.104564              0.999166       0.9842                    519.574       
     total [########################################..........] 80.00%
this epoch [..................................................]  0.00%
     24000 iter, 40 epoch / 50 epochs
    46.091 iters/sec. Estimated time to finish: 0:02:10.177932.
     total [########################################..........] 80.33%
this epoch [########..........................................] 16.67%
     24100 iter, 40 epoc

     total [############################################......] 89.67%
this epoch [#########################################.........] 83.33%
     26900 iter, 44 epoch / 50 epochs
    46.304 iters/sec. Estimated time to finish: 0:01:06.948509.
45          0.00373737  0.161767              0.998899       0.9799                    584.246       
     total [#############################################.....] 90.00%
this epoch [..................................................]  0.00%
     27000 iter, 45 epoch / 50 epochs
    46.152 iters/sec. Estimated time to finish: 0:01:05.002452.
     total [#############################################.....] 90.33%
this epoch [########..........................................] 16.67%
     27100 iter, 45 epoch / 50 epochs
    46.157 iters/sec. Estimated time to finish: 0:01:02.829226.
     total [#############################################.....] 90.67%
this epoch [################..................................] 33.33%
     27200 iter, 45 epoc

50          0.00295057  0.130484              0.9991         0.9829                    649.201       
     total [##################################################] 100.00%
this epoch [..................................................]  0.00%
     30000 iter, 50 epoch / 50 epochs
    46.182 iters/sec. Estimated time to finish: 0:00:00.
